In [1]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [2]:
train = pd.read_csv("../input/train_F3WbcTw.csv")
test = pd.read_csv("../input/test_tOlRoBf.csv")

In [3]:
train.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,"Autoimmune diseases tend to come in clusters. As for Gilenya – if you feel good, don’t think about it, it won’t change anything but waste your time and energy. I’m taking Tysabri and feel amazing,...",gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,"I can completely understand why you’d want to try it. But, results reported in lectures don’t always stand up to the scrutiny of peer-review during publication. There so much still to do before th...",gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,"Interesting that it only targets S1P-1/5 receptors rather than 1-5 like Fingolimod. Hoping to soon see what the AEs and SAEs were Yes. I'm not sure what this means, exactly: Quote Nine patients r...",fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder where lemtrada and ocrevus sales would go, if they prove anti-cd20 are induction",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain and Cervical Cord are in and my next Neurologist appointment is in the next couple of weeks. There’re no new lesions in Brain/Cord and I’ve had no rel...",gilenya,1


In [4]:
train['sentiment'].value_counts(normalize = True)

2    0.724569
1    0.158553
0    0.116878
Name: sentiment, dtype: float64

In [5]:
train['text'] = train['text'] + "This observation is for " + train['drug']
test['text'] = test['text'] + "This observation is for " + test['drug']

In [6]:
import re
def pre_process(text):
    new_text =re.sub('[0-9]', '', text)
    new_text = re.sub(r"\u200b","",new_text)
    new_text = re.sub(r"\.+",".",new_text)
    new_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '',new_text, flags=re.MULTILINE)
    new_text = re.sub("'", "", new_text)
    new_text = re.sub(r'↑', '', new_text)
    new_text = re.sub("\t", "", new_text)
    new_text = re.sub("\xa0", "", new_text)
    new_text = re.sub("\(|\)|\[|\]", "", new_text)
    new_text = re.sub("\n", "", new_text)
    new_text = re.sub("\.", "", new_text)
    new_text = re.sub("\,", " ", new_text)
    new_text = re.sub("[/%]", " ", new_text)
    new_text = re.sub('[/%:;]', '', new_text)
    new_text = re.sub(' +', ' ', new_text)
    return new_text

In [7]:
%%time

# remove URL's from train and test
for index, row in train['text'].iteritems():
    train['text'][index] = pre_process(row)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 3min 6s, sys: 48 ms, total: 3min 6s
Wall time: 3min 6s


In [8]:
%%time 

for index, row in test['text'].iteritems():
    test['text'][index] = pre_process(row)

CPU times: user 1.82 s, sys: 24 ms, total: 1.85 s
Wall time: 1.82 s


In [9]:
# remove URL's from train and test
train['text'] = train['text'].apply(lambda x: re.sub(r'http\S+', '', x))

test['text'] = test['text'].apply(lambda x: re.sub(r'http\S+', '', x))

In [10]:
# remove numbers
train['text'] = train['text'].str.replace("[0-9]", " ")
test['text'] = test['text'].str.replace("[0-9]", " ")

In [11]:
# Handle Won't 

replaceWords1 = { "won't":"will not","$&@*#":"in most profane vulgar shitty terms","#$&@*#":"shitty",
 "can't":"cannot","aren't": 'are not',
 "Aren't": 'Are not',
 "AREN'T": 'ARE NOT',
 "C'est": "C'est",
 "C'mon": "C'mon",
 "c'mon": "c'mon",
 "can't": 'cannot',
 "Can't": 'Cannot',
 "CAN'T": 'CANNOT',
 "con't": 'continued',
 "cont'd": 'continued',
 "could've": 'could have',
 "couldn't": 'could not',
 "Couldn't": 'Could not',
 "didn't": 'did not',
 "Didn't": 'Did not',
 "DIDN'T": 'DID NOT',
 "don't": 'do not',
 "Don't": 'Do not',
 "DON'T": 'DO NOT',
 "doesn't": 'does not',
 "Doesn't": 'Does not',
 "else's": 'else',
 "gov's": 'government',
 "Gov's": 'government',
 "gov't": 'government',
 "Gov't": 'government',
 "govt's": 'government',
 "gov'ts": 'governments',
 "hadn't": 'had not',
 "hasn't": 'has not',
 "Hasn't": 'Has not',
 "haven't": 'have not',
 "Haven't": 'Have not',
 "he's": 'he is',
 "He's": 'He is',
 "he'll": 'he will',
 "He'll": 'He will',
 "he'd": 'he would',
 "He'd": 'He would',
 "Here's": 'Here is',
 "here's": 'here is',
 "I'm": 'I am',
 "i'm": 'i am',
 "I'M": 'I am',
 "I've": 'I have',
 "i've": 'i have',
 "I'll": 'I will',
 "i'll": 'i will',
 "I'd": 'I would',
 "i'd": 'i would',
 "ain't": 'is not',
 "isn't": 'is not',
 "Isn't": 'Is not',
 "ISN'T": 'IS NOT',
 "it's": 'it is',
 "It's": 'It is',
 "IT'S": 'IT IS',
 "I's": 'It is',
 "i's": 'it is',
 "it'll": 'it will',
 "It'll": 'It will',
 "it'd": 'it would',
 "It'd": 'It would',
 "Let's": "Let's",
 "let's": 'let us',
 "ma'am": 'madam',
 "Ma'am": "Madam",
 "she's": 'she is',
 "She's": 'She is',
 "she'll": 'she will',
 "She'll": 'She will',
 "she'd": 'she would',
 "She'd": 'She would',
 "shouldn't": 'should not',
 "that's": 'that is',
 "That's": 'That is',
 "THAT'S": 'THAT IS',
 "THAT's": 'THAT IS',
 "that'll": 'that will',
 "That'll": 'That will',
 "there's": 'there is',
 "There's": 'There is',
 "there'll": 'there will',
 "There'll": 'There will',
 "there'd": 'there would',
 "they're": 'they are',
 "They're": 'They are',
 "they've": 'they have',
 "They've": 'They Have',
 "they'll": 'they will',
 "They'll": 'They will',
 "they'd": 'they would',
 "They'd": 'They would',
 "wasn't": 'was not',
 "we're": 'we are',
 "We're": 'We are',
 "we've": 'we have',
 "We've": 'We have',
 "we'll": 'we will',
 "We'll": 'We will',
 "we'd": 'we would',
 "We'd": 'We would',
 "What'll": 'What will',
 "weren't": 'were not',
 "Weren't": 'Were not',
 "what's": 'what is',
 "What's": 'What is',
 "When's": 'When is',
 "Where's": 'Where is',
 "where's": 'where is',
 "Where'd": 'Where would',
 "who're": 'who are',
 "who've": 'who have',
 "who's": 'who is',
 "Who's": 'Who is',
 "who'll": 'who will',
 "who'd": 'Who would',
 "Who'd": 'Who would',
 "won't": 'will not',
 "Won't": 'will not',
 "WON'T": 'WILL NOT',
 "would've": 'would have',
 "wouldn't": 'would not',
 "Wouldn't": 'Would not',
 "would't": 'would not',
 "Would't": 'Would not',
 "y'all": 'you all',
 "Y'all": 'You all',
 "you're": 'you are',
 "You're": 'You are',
 "YOU'RE": 'YOU ARE',
 "you've": 'you have',
 "You've": 'You have',
 "y'know": 'you know',
 "Y'know": 'You know',
 "ya'll": 'you will',
 "you'll": 'you will',
 "You'll": 'You will',
 "you'd": 'you would',
 "You'd": 'You would',
 "Y'got": 'You got',
 'cause': 'because',
 "had'nt": 'had not',
 "Had'nt": 'Had not',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how is',
 "I'd've": 'I would have',
 "I'll've": 'I will have',
 "i'd've": 'i would have',
 "i'll've": 'i will have',
 "it'd've": 'it would have',
 "it'll've": 'it will have',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "o'clock": 'of the clock',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "shan't": 'shall not',
 "sha'n't": 'shall not',
 "shan't've": 'shall not have',
 "she'd've": 'she would have',
 "she'll've": 'she will have',
 "should've": 'should have',
 "shouldn't've": 'should not have',
 "so've": 'so have',
 "so's": 'so as',
 "this's": 'this is',
 "that'd": 'that would',
 "that'd've": 'that would have',
 "there'd've": 'there would have',
 "they'd've": 'they would have',
 "they'll've": 'they will have',
 "to've": 'to have',
 "we'd've": 'we would have',
 "we'll've": 'we will have',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "what're": 'what are',
 "what've": 'what have',
 "when's": 'when is',
 "when've": 'when have',
 "where'd": 'where did',
 "where've": 'where have',
 "who'll've": 'who will have',
 "why's": 'why is',
 "why've": 'why have',
 "will've": 'will have',
 "won't've": 'will not have',
 "wouldn't've": 'would not have',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "you'd've": 'you would have',
 "you'll've": 'you will have',
'bebecause':'be because',
'I’m':'I am',
              'it’s':'it is',
                 'I’ve':'I have',
                 'don’t':'do not',
                'However':'but',
                 'It’s':'It is',
                 'didn’t':'did not',
                 'can’t':'can not',
                 'that’s':'that is',
'doesn’t':'does not',
'I’d':'I had',
'isn’t':'is not',
'wasn’t':'was not'
                
                }

def wordreplace(tweet,replaceWords):
    for key in replaceWords:
        tweet = tweet.replace(key,replaceWords[key])
    return tweet

for index, row in train['text'].iteritems():
    train['text'][index] = wordreplace(row,replaceWords1)
    
for index, row in test['text'].iteritems():
    test['text'][index] = wordreplace(row,replaceWords1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
import string
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list += "'"

In [13]:
#train['text'] = train['text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(white_list)))
#test['text'] = test['text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(white_list)))

In [14]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['text'] = train['text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['text'] = test['text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

In [15]:
# remove whitespaces
train['text'] = train['text'].apply(lambda x:' '.join(x.split()))
test['text'] = test['text'].apply(lambda x: ' '.join(x.split()))

In [16]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [17]:
train['text'] = lemmatization(train['text'])
test['text'] = lemmatization(test['text'])

In [18]:
X = train['text']
y = train['sentiment']

In [19]:
X.shape

(5279,)

In [20]:
from sklearn.model_selection import train_test_split

x_train , x_test, y_train  , y_test = train_test_split(X , 
                                                     y , 
                                                      stratify = y.values , 
                                                     train_size = 0.8,
                                                     random_state = 100)

## Embeddings

In [21]:
tfhub_dir = '/data/jupyter/common/model/text/tfhub'

import sys, os
def add_aion(curr_path=None):
    if curr_path is None:
        dir_path = os.getcwd()
        target_path = os.path.dirname(dir_path)
        if target_path not in sys.path:
            print('Added %s into sys.path.' % (target_path))
            sys.path.insert(0, target_path)
            
add_aion()

Added /kaggle into sys.path.


## Model

In [22]:
vocab = set()
for sentence in x_train:
    tokens = sentence.split(' ')
    for token in tokens:
        vocab.add(token)

vocab_size = len(vocab)
print('Vocab Size: %d' % (vocab_size))

Vocab Size: 43111


In [23]:
max_sentence_length = 270
word2Idx = {'<padding>': 0, '<unknown>': 1}
idx2word = {0: '<padding>', 1: '<unknown>'}

def preprocess(text, word2Idx, idx2word, training=False):
    if training:
        for sentence in text:
            tokens = sentence.split(' ')

            for token in tokens:
                if token not in word2Idx:
                    word2Idx[token] = len(word2Idx)
                    idx2word[len(word2Idx)-1] = token


    word_vectors = np.zeros((len(text), max_sentence_length))
    sentence_vectors = []
    
    for i, sentence in enumerate(text):
        ids = []
        words = []
        tokens = sentence.split(' ')
        for token in tokens:
            if token in word2Idx:
                ids.append(word2Idx[token])
                words.append(token)
            else:
                ids.append(word2Idx['<unknown>'])
                words.append('<unknown>')
                
            if len(ids) >= max_sentence_length:
                break

        for i in range(max_sentence_length - len(ids)):
            ids.append(word2Idx['<padding>'])
            words.append('<padding>')

        word_vectors[i] = np.asarray(ids)
        sentence_vectors.append(' '.join(words))

    sentence_vectors = np.asarray(sentence_vectors)
    
    return word2Idx, idx2word, word_vectors, sentence_vectors

In [24]:
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

class ELMoEmbedding(Layer):

    def __init__(self, idx2word, output_mode="default", trainable=True, **kwargs):
        assert output_mode in ["default", "word_emb", "lstm_outputs1", "lstm_outputs2", "elmo"]
        assert trainable in [True, False]
        self.idx2word = idx2word
        self.output_mode = output_mode
        self.trainable = trainable
        self.max_length = None
        self.word_mapping = None
        self.lookup_table = None
        self.elmo_model = None
        self.embedding = None
        super(ELMoEmbedding, self).__init__(**kwargs)

    def build(self, input_shape):
        self.max_length = input_shape[1]
        self.word_mapping = [x[1] for x in sorted(self.idx2word.items(), key=lambda x: x[0])]
        self.lookup_table = tf.contrib.lookup.index_to_string_table_from_tensor(self.word_mapping, default_value="<UNK>")
        self.lookup_table.init.run(session=K.get_session())
        self.elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=self.trainable)
        super(ELMoEmbedding, self).build(input_shape)

    def call(self, x):
        x = tf.cast(x, dtype=tf.int64)
        sequence_lengths = tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32)
        strings = self.lookup_table.lookup(x)
        inputs = {
            "tokens": strings,
            "sequence_len": sequence_lengths
        }
        return self.elmo_model(inputs, signature="tokens", as_dict=True)[self.output_mode]

    def compute_output_shape(self, input_shape):
        if self.output_mode == "default":
            return (input_shape[0], 1024)
        if self.output_mode == "word_emb":
            return (input_shape[0], self.max_length, 512)
        if self.output_mode == "lstm_outputs1":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "lstm_outputs2":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "elmo":
            return (input_shape[0], self.max_length, 1024)

    def get_config(self):
        config = {
            'idx2word': self.idx2word,
            'output_mode': self.output_mode 
        }
        return list(config.items())

Using TensorFlow backend.


In [25]:
word2Idx, idx2word, x_train_words, x_train_sentences = preprocess(
    text=x_train, word2Idx=word2Idx, idx2word=idx2word, training=True)
print('x_train_words.shape:', x_train_words.shape)

word2Idx, idx2word, x_test_words, x_test_sentences = preprocess(
    text=x_test, word2Idx=word2Idx, idx2word=idx2word, training=False)
print('x_test_words.shape:', x_test_words.shape)

x_train_words.shape: (4223, 270)
x_test_words.shape: (1056, 270)


## Weighted sum of the 3 layers with word embeddings

In [26]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Dense, Embedding, BatchNormalization, Concatenate, LSTM
from keras.models import Model

In [27]:
import datetime, os, urllib, zipfile


class Embeddings:
    def __init__(self, verbose=0):
        self.verbose = verbose
        self.model = {}
        self.model_path = ''
        
    def _log_time(self, status, msg, verbose):
        if self.verbose >= 10 or verbose >= 10:
            print('%s. [%s] %s' % (datetime.datetime.now(), status, msg))
            
    def download(self, src, dest_dir, dest_file, uncompress, housekeep=False, verbose=0):
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
    
        if dest_file is None:
            dest_file = os.path.basename(src)
            
        if not self.is_file_exist(dest_dir + dest_file):
            self._log_time(status='DOWNLOAD', msg='From '+src+' to '+dest_dir+dest_file, verbose=verbose)
            file = urllib.request.urlopen(src)
            with open(dest_dir + dest_file,'wb') as output:
                output.write(file.read())
        else:
            self._log_time(status='FOUND', msg=dest_file+' in '+dest_dir, verbose=verbose)
            
        if uncompress:
            self.uncompress(dest_dir + dest_file)
            
        if uncompress and housekeep:
            self.housekeep(dest_dir + dest_file)
            
            
        return dest_dir + dest_file
    
    """
        File related
    """
    
    def uncompress(self):
        raise NotImplemented()
    
    def unzip(self, file_path):
        dest_dir = os.path.dirname(file_path)
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(dest_dir)
            
    def housekeep(self, file_path):
        os.remove(file_path)
        
    def is_file_exist(self, file_path):
        return os.path.exists(file_path)
    
    def save(self):
        raise NotImplemented()
        
    def load(self):
        raise NotImplemented()
    
    """
        Model related
    """
        
    def get_model(self):
        return self.model
    
    def set_model(self, model):
        self.model = model
    
    def load(self, src=None, dest_dir=None, trainable=True, verbose=0):
        raise NotImplemented()
        
    """
        Vocabulary realted
    """
    
    def load_vocab(self, **kwargs):
        raise NotImplemented()
        
    def build_vocab(self):
        raise NotImplemented()
    
    def get_vocab(self):
        raise NotImplemented()
        
    def _tokenizer_space(self, sentence):
        return sentence.split(' ')
        
    """
        Vector related
    """
    
    def train(self):
        raise NotImplemented()
        
    def encode(self, sentences):
        raise NotImplemented()
        
    def visualize(self):
        raise NotImplemented()
        
    """
        Netowrk realted
    """
    
    def to_numpy_layer(self):
        raise NotImplemented()
        
    def to_keras_layer(self):
        raise NotImplemented()
        
    def to_tensorflow_layer(self):
        raise NotImplemented()
    
    def to_pytorch_layer(self):
        raise NotImplemented()

In [28]:
class WordEmbeddings(Embeddings):
    
    def __init__(self, 
                 handle_oov=True, oov_vector=None, oov_vector_type='zero',
                 padding=True, pad_vector=None, pad_vector_type='zero',
                 max_sequence_length=350, dimension=350,
                 verbose=0):
        super().__init__(verbose=verbose)
        self.handle_oov = handle_oov
        self.oov_vector_type = oov_vector_type
        if handle_oov and oov_vector is None:
            if oov_vector_type == 'zero':
                self.oov_vector = np.zeros(dimension)
            elif oov_vector_type == 'random':
                self.oov_vector = np.random.rand(dimension)
        else:
            self.oov_vector = oov_vector
            
        self.padding = padding
        self.pad_vector_type = pad_vector_type
        if padding and pad_vector is None:
            if pad_vector_type == 'zero':
                self.pad_vector = np.zeros(dimension)
            elif pad_vector_type == 'random':
                self.pad_vector = np.random.rand(dimension)
        else:
            self.pad_vector = pad_vector
        
        self.max_sequence_length = max_sequence_length
        self.dimension = dimension
        
    def get_oov_vector(self):
        return self.oov_vector
        
    def set_oov_vector(self, oov_vector):
        self.oov_vector = oov_vector
        
    def get_pad_vector(self):
        return self.pad_vector
        
    def set_pad_vector(self, pad_vector):
        self.pad_vector = pad_vector
        
    def is_vector_exist(self, word):
        return word in self.model

In [29]:
import datetime
import os
import tensorflow as tf
import tensorflow_hub as tf_hub


class ELMoEmbeddings(WordEmbeddings):
    ELMO_MODEL_V2_URL = "https://tfhub.dev/google/elmo/2"

    def __init__(self, layer, verbose=0):
        super().__init__(verbose=verbose)
        self.layer = layer
        
    def _set_tf_log_level(self, verbose):
        if verbose >= 30:
            tf.logging.set_verbosity(tf.logging.INFO)
        elif verbose >= 20:
            tf.logging.set_verbosity(tf.logging.WARN)
        elif verbose >= 10:
            tf.logging.set_verbosity(tf.logging.DEBUG)
        else:
            tf.logging.set_verbosity(tf.logging.ERROR)
        
    def load(self, src=None, dest_dir=None, trainable=True, verbose=0):
        self._log_time(status='LOADING', msg='file', verbose=verbose)
        self._set_tf_log_level(verbose)
        
        if src == None:
            src = self.ELMO_MODEL_V2_URL
        
        if dest_dir is not None:
            os.environ["TFHUB_CACHE_DIR"] = dest_dir
        
        self.model = tf_hub.Module(src, trainable=trainable)
        
        self._log_time(status='LOADED', msg='', verbose=verbose)
        
        return self.model    
    
    def to_keras_layer(self, x):
        # Source: https://github.com/strongio/keras-elmo/blob/master/Elmo%20Keras.ipynb
        '''
            For signature and layer parameters, you can visit https://alpha.tfhub.dev/google/elmo/2
        '''        
        return self.model(
            tf.squeeze(tf.cast(x, tf.string)), 
            signature="default", as_dict=True)[self.layer]

In [30]:
elmo_embs = ELMoEmbeddings(layer='elmo', verbose=20)
elmo_embs.load(dest_dir=tfhub_dir)

2019-07-28 16:18:34.664078. [LOADING] file
2019-07-28 16:18:55.597850. [LOADED] 


In [31]:
from keras.callbacks import EarlyStopping 
es = EarlyStopping(monitor='val_loss', mode ='min' ,verbose =1,patience=2)

In [32]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [33]:
# Input Layers
word_input_layer = Input(shape=(None, ), dtype='int32')
elmo_input_layer = Input(shape=(None, ), dtype=tf.string)

# Output Layers
word_output_layer = Embedding(
    input_dim=vocab_size, output_dim=256)(word_input_layer)
elmo_output_layer = Lambda(
    elmo_embs.to_keras_layer, 
    output_shape=(None, 1024))(elmo_input_layer)
output_layer = Concatenate()(
    [word_output_layer, elmo_output_layer])
output_layer = BatchNormalization()(output_layer)
output_layer = LSTM(
    256, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(output_layer)
output_layer = LSTM(
    256, dropout=0.2, recurrent_dropout=0.2)(output_layer)
output_layer = Dense(100, activation='relu')(output_layer)
output_layer = Dense(3, activation='softmax')(output_layer)

# Build Model
model = Model(
    inputs=[word_input_layer, elmo_input_layer], 
    outputs=output_layer)
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(
    [x_train_words, x_train_sentences], y_train,
     validation_data=([x_test_words, x_test_sentences], y_test), 
     callbacks=[es],
    class_weight=class_weights,
    epochs=100, batch_size=100)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    11036416    input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 1024)   0           input_2[0][0]                    
__________________________________________________________________________________________________
concatenat

In [34]:
y_pred = model.predict([x_test_words, x_test_sentences])
y_pred = np.argmax(y_pred, axis=1)

In [35]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       123
           1       0.00      0.00      0.00       168
           2       0.72      1.00      0.84       765

    accuracy                           0.72      1056
   macro avg       0.24      0.33      0.28      1056
weighted avg       0.52      0.72      0.61      1056



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
word2Idx, idx2word, test_words, test_sentences = preprocess(
    text=test['text'], word2Idx=word2Idx, idx2word=idx2word, training=True)
print('test_words.shape:', test_words.shape)

test_words.shape: (2924, 270)


In [37]:
predictions=model.predict([test_words,test_sentences])

In [38]:
prediction_round = np.argmax(predictions,axis=1)
sub = pd.read_csv("../input/sample_submission_i5xnIZD.csv")
sub['sentiment'] = prediction_round
sub.to_csv("ELMO_1L.csv",index=False)

## Weighted sum of the 3 layers without word embeddings

In [39]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Dense, Embedding, BatchNormalization, Concatenate, LSTM
from keras.models import Model

elmo_embs = ELMoEmbeddings(layer='elmo', verbose=20)
elmo_embs.load(dest_dir=tfhub_dir)

2019-07-28 16:28:20.516427. [LOADING] file
2019-07-28 16:28:21.395307. [LOADED] 


In [40]:
# Input Layers
elmo_input_layer = Input(shape=(None, ), dtype=tf.string)

# Output Layers
output_layer = Lambda(
    elmo_embs.to_keras_layer, 
    output_shape=(None, 1024))(elmo_input_layer)
output_layer = BatchNormalization()(output_layer)
output_layer = LSTM(
    256, dropout=0.2, recurrent_dropout=0.2)(output_layer)
output_layer = Dense(3, activation='softmax')(output_layer)

# Build Model
model = Model(
    inputs=elmo_input_layer, outputs=output_layer)
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(
    x_train_sentences, y_train,
    validation_data=(x_test_sentences, y_test), 
    callbacks=[es],
    class_weight=class_weights,
    epochs=10, batch_size=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None)              0         
_________________________________________________________________
lambda_2 (Lambda)            (None, None, 1024)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 1024)        4096      
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               1311744   
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total params: 1,316,611
Trainable params: 1,314,563
Non-trainable params: 2,048
_________________________________________________________________
Train on 4223 samples, validate on 1056 samples
Epoch 1/10
4223/4223 [==============================] - 130s 31ms/step - loss: 0

In [41]:
y_pred = model.predict(x_test_sentences)
y_pred = np.argmax(y_pred, axis=1)

In [42]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       123
           1       0.50      0.01      0.02       168
           2       0.72      0.99      0.84       765

    accuracy                           0.72      1056
   macro avg       0.41      0.33      0.29      1056
weighted avg       0.60      0.72      0.61      1056



In [43]:
predictions=model.predict(test_sentences)

In [44]:
prediction_round = np.argmax(predictions,axis=1)

In [45]:
sub = pd.read_csv("../input/sample_submission_i5xnIZD.csv")
sub['sentiment'] = prediction_round
sub.to_csv("ELMO_2L.csv",index=False)

## Fixed Mean-pooling without word embeddings¶

In [46]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Dense, Embedding, BatchNormalization, Concatenate, LSTM
from keras.models import Model

elmo_embs = ELMoEmbeddings(layer='default', verbose=20)
elmo_embs.load(dest_dir=tfhub_dir)

2019-07-28 16:36:38.322531. [LOADING] file
2019-07-28 16:36:39.100433. [LOADED] 


In [47]:
# Input Layers
input_layer = Input(shape=(None,), dtype=tf.string)

# Output Layers
output_layer = Lambda(
    elmo_embs.to_keras_layer, 
    output_shape=(1024,))(input_layer)
output_layer = Dense(
    256, activation='relu')(output_layer)
output_layer = Dense(3, activation='softmax')(output_layer)

model = Model(inputs=[input_layer], outputs=output_layer)
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', metrics=['accuracy'])
model.summary()

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
model.fit(
    x_train_sentences, y_train,
    validation_data=(x_test_sentences, y_test), 
    callbacks=[es],
    epochs=10, batch_size=32,class_weight=class_weights)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None)              0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 771       
Total params: 263,171
Trainable params: 263,171
Non-trainable params: 0
_________________________________________________________________
Train on 4223 samples, validate on 1056 samples
Epoch 1/10
4223/4223 [==============================] - 138s 33ms/step - loss: 0.7913 - acc: 0.7156 - val_loss: 0.7656 - val_acc: 0.7244
Epoch 2/10
4223/4223 [==============================] - 135s 32ms/step - loss: 0.75

In [48]:
y_pred = model.predict(x_test_sentences)
y_pred = np.argmax(y_pred, axis=1)

In [49]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.01      0.02       123
           1       0.00      0.00      0.00       168
           2       0.73      1.00      0.84       765

    accuracy                           0.72      1056
   macro avg       0.33      0.34      0.29      1056
weighted avg       0.55      0.72      0.61      1056



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [50]:
predictions=model.predict(test_sentences)
prediction_round = np.argmax(predictions,axis=1)
sub = pd.read_csv("../input/sample_submission_i5xnIZD.csv")
sub['sentiment'] = prediction_round
sub.to_csv("ELMO_3L.csv",index=False)